In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Extract Data

In [ ]:
from bs4 import BeautifulSoup 
entries = []
entry = []
urlnumber = 2
while urlnumber < 105: 
    url = 'https://forums.edmunds.com/discussion/7526/general/x/midsize-sedans-2-0/p%d' % (urlnumber,)
    try:
        r = requests.get(url, timeout = 10) 
    except Exception as e:
        print("Error message:",e)
        break;

    data = r.text
    
    soup = BeautifulSoup(data, 'lxml')
    
    for div in soup.find_all('div'):
        entry = []
        if(div.get('class') != None and div.get('class')[0] == 'Comment'): 
            ps = div.find_all('p') 
            aas = div.find_all('a')
            spans = div.find_all('span')
            times = div.find_all('time') 
            concat_str = ''
            for str in aas[1].contents: 
                if str != "<br>" or str != "<br/>": 
                    concat_str = (concat_str + ' '+ str).encode("utf-8").strip()
            entry.append(concat_str)

            concat_str = ''
            for str in times[0].contents:
                if str != "<br>" or str != "<br/>":
                    concat_str = (concat_str + ' '+ str).encode('iso-8859-1').strip()
            entry.append(concat_str)

            for div in div.find_all('div'):
                if (div.get('class') != None and div.get('class')[0] == 'Message'): 
                    blockquotes = []
                    x = div.get_text()
                    for bl in div.find_all('blockquote'):
                        blockquotes.append(bl.get_text()) 
                        bl.decompose()
                    ascii_encoding = div.get_text().replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                    latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                    entry.append(latin1_encoding)

                    for bl in blockquotes:
                        ascii_encoding = bl.replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                        latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                        entry.append(latin1_encoding)

            entries.append(entry)
            
    urlnumber += 1

stringlist=[[x.decode('iso-8859-1') for x in entry] for entry in entries]
with open('edmunds_extraction.csv', 'w') as output:
    writer = csv.writer(output, quoting=csv.QUOTE_ALL)
    writer.writerows(stringlist)

print ("Wrote to edmunds_extraction.csv")

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('edmunds_extraction.csv',header=None)

In [ ]:
df.columns=['userid','date','message']

In [ ]:
df

### Brands and models from external source plus models.csv file given

In [ ]:
df_brands=pd.read_csv('brands.csv')
df_models=pd.read_csv('model.csv')
df_brand_model=pd.merge(df_models,df_brands,left_on='brand_id',right_on='id')
df_brand_model=df_brand_model[['name_x','name_y']].rename(columns={'name_x':'Model','name_y':'Brand'})
df_brand_model_given=pd.read_csv('models.csv',header=None).rename(columns={0:'Brand',1:'Model'})
df_brand_model['Model']=df_brand_model['Model'].apply(lambda x:x.lower())
df_brand_model['Brand']=df_brand_model['Brand'].apply(lambda x:x.lower())
df_brand_model_given['Model']=df_brand_model_given['Model'].apply(lambda x:x.lower())
df_brand_model_given['Brand']=df_brand_model_given['Brand'].apply(lambda x:x.lower())
df_brand_model_final=df_brand_model_given.append(df_brand_model).drop_duplicates()
df_brand_model=df_brand_model_final.reset_index().drop(columns='index')
df_brand_model

In [ ]:
df['message']=df['message'].astype('str')

In [ ]:
#df=df[1:50]

### Stopwords removal

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['message'] = df['message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df)

### Tokenization

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
df['message']=df['message'].apply(lambda x:word_tokenize(x))

### Tagging words with pos_tag and lemmatization
- Extracted only the nouns from pos tag as brands and models will come under them
- Didn't use porter stemmer as it can manipulate the brand names

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
df['message']=df['message'].apply(lambda x:nltk.pos_tag(x))

In [ ]:
for l in range(len(df['message'])):
    df['message'].iloc[l]=[j[0] for j in df['message'].iloc[l] if j[1].startswith("NN")]

In [ ]:
df['message']

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w).lower() for w in text]
df['message']=df['message'].apply(lambda x:lemmatize_text(x))

from nltk.stem import PorterStemmer
ps = PorterStemmer()
def porter_stemmer(text):
    return [ps.stem(w).lower() for w in text]
#df['message'].apply(lambda x:porter_stemmer(x))

In [ ]:
df

In [ ]:
df['message']

In [ ]:
df_brand_model

In [ ]:
for i in range(len(df['message'])):
    for j in range(len(df_brand_model)):
        if(df_brand_model['Model'].iloc[j] in df['message'].iloc[i]):
            print(df_brand_model['Model'].iloc[j])
            idx=df['message'].iloc[i].index(df_brand_model['Model'].iloc[j])
            df['message'].iloc[i][idx]=df_brand_model['Brand'].iloc[j]

In [ ]:
list(df_brand_model['Model'])

In [ ]:
df_brand_model['Brand']=df_brand_model['Brand'].str.replace(',','')
df_brand_model['Model']=df_brand_model['Model'].str.replace('.','')
df_brand_model['Brand']=df_brand_model['Brand'].str.replace('.','')
df_brand_model['Model']=df_brand_model['Model'].str.replace(',','')
df_brand_model['Brand']=df_brand_model['Brand'].apply(lambda x:x.strip())
df_brand_model['Model']=df_brand_model['Model'].apply(lambda x:x.strip())

In [ ]:
for brand in set(df_brand_model['Brand']):
    df[brand]=np.nan

In [ ]:
df

In [ ]:
for brand in set(df_brand_model['Brand']):
    for j in range(len(df['message'])):
        if(brand in df['message'].iloc[j]):
            df[brand].iloc[j]=1
        else:
            df[brand].iloc[j]=0

In [ ]:
df.iloc[:, 3:].sum(axis=1)

### Top 10 brands by frequency

In [ ]:
df_top_ten_brands=pd.DataFrame(df.iloc[:, 3:].sum(axis=0)).reset_index()
df_top_ten_brands.columns=['brand','frequency']
df_top_ten_brands=df_top_ten_brands.sort_values(by='frequency',ascending=False).reset_index().drop(columns=['index'])

In [ ]:
list(df_top_ten_brands['brand'][2:12])

In [ ]:
df[df['sum']>0]

### Lift Similarity of top 10 brands

In [ ]:
df_1=df.iloc[:,3:]

In [ ]:
df_1=df_1[['honda',
 'ford',
 'toyota',
 'hyundai',
 'nissan',
 'mazda',
 'sedan',
 'problem',
 'chevrolet',
 'chrysler']]

In [ ]:
df_1

In [ ]:
#df_1=df_1.drop(columns=['sum'])

In [ ]:
#df_1=df_1[df_1.columns[df_1.sum()>0]]

In [ ]:
#len(df_1[(df_1[df_1.columns[0]]==1) & (df_1[df_1.columns[1]]==1)])/len(df_1[(df_1[df_1.columns[1]]==1)])

In [ ]:
df_1=df_1.reset_index().drop(columns='index')

In [ ]:
df_1

In [ ]:
idx=0
for i in range(len(df_1.columns)):
    for j in range(i+1,len(df_1.columns)):
        #df_1[df_1.columns[i+1]].iloc[idx]=99
        print(i,j,df_1.columns[i],df_1.columns[j],len(df_1[(df_1[df_1.columns[i]]==1) & (df_1[df_1.columns[j]]==1)])*len(df_1)/(len(df_1[(df_1[df_1.columns[i]]==1)])*len(df_1[(df_1[df_1.columns[j]]==1)])))
        df_1[df_1.columns[j]].iloc[idx]=len(df_1[(df_1[df_1.columns[i]]==1) & (df_1[df_1.columns[j]]==1)])*len(df_1)/(len(df_1[(df_1[df_1.columns[i]]==1)])*len(df_1[(df_1[df_1.columns[j]]==1)]))
    idx=idx+1
    

In [ ]:
df_1=df_1[:len(df_1.columns)]

In [ ]:
cols=df_1.columns

In [ ]:
m,n = df_1.shape
df_1=pd.DataFrame(np.where(np.arange(m)[:,None] >= np.arange(n),np.nan,df_1))
df_1=df_1.fillna(0)

In [ ]:
df_1.index=cols
df_1.columns=cols

In [ ]:
df_1

### 1/Lift(dissimilarity)

In [ ]:
df_2=pd.DataFrame()
for l in df_1.columns:
    df_2[l]=np.nan
    df_2[l]=df_1[l].astype('int')
    df_2[l]=1/df_1[l]

In [ ]:
df_2.columns

In [ ]:
df_2